In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from config import username, password

## Extract CSV Files

In [ ]:
mcd_file = "Resources/mcd_menu.csv"
bk_mcd_file = "Resources/bk_mcd_menu.csv"
starbucks_food_file = "Resources/starbucks_food.csv"
starbucks_drink_file = "Resources/starbucks_drink_menu.csv"
subway_file = "Resources/subway_menu.csv"

In [ ]:
mcd_df = pd.read_csv(mcd_file)
bk_mcd_df = pd.read_csv(bk_mcd_file, delimiter=';')
starbucks_food_df = pd.read_csv(starbucks_food_file)
starbucks_drink_df = pd.read_csv(starbucks_drink_file)
subway_df = pd.read_csv(subway_file)

## Transform Data

### Transform Subway Data

In [ ]:
subway_df.head()

In [ ]:
# extract columns desired for database
subway_transformed = subway_df[["Category", "Unnamed: 0", "Saturated Fat (g)", "Calories"]].copy()

# rename columns
subway_transformed.rename(columns={"Category": "category", 
                                   "Unnamed: 0": "item",
                                   "Saturated Fat (g)": "saturated_fat",
                                   "Calories": "calories"}, inplace=True)

# add "food_class" column
# recognizing that category == Extra are desserts in the dataset
conditions = [(subway_transformed["category"] == "Extra")]

values = [1]

subway_transformed["food_class"] = np.select(conditions, values)

subway_transformed["food_class"].replace(0,3, inplace=True)

# create "id" column
subway_transformed["id"] = subway_transformed.index

subway_transformed.set_index("id", inplace=True)

# display dataframe
subway_transformed.head()

### Transform Burger King and McDonalds Data

In [ ]:
mcd_df.head()

In [ ]:
# Create a filtered dataframe from specific columns
mcd_cols = ["Category", "Item", "Saturated Fat", "Calories"]
mcd_transformed= mcd_df[mcd_cols].copy()

# Rename the column headers for consistency
mcd_transformed = mcd_transformed.rename(columns={"Category": "category",
                                                    "Item": "item",
                                                    "Saturated Fat": "saturated_fat",
                                                    "Calories": "calories"})

mcd_transformed.head()

In [ ]:
# Find full list of categories
mcd_transformed['category'].unique()

In [ ]:
# manually assign each category to a food_class 
conditions = [(mcd_transformed['category'] == 'Breakfast') | (mcd_transformed['category'] == 'Beef & Pork') \
                  | (mcd_transformed['category'] == 'Chicken & Fish') | (mcd_transformed['category'] == 'Salads') \
                  | (mcd_transformed['category'] == 'Snacks & Sides'),
              (mcd_transformed['category'] == 'Beverages') | (mcd_transformed['category'] == 'Smoothies & Shakes') \
                  | (mcd_transformed['category'] == 'Coffee & Tea'),
              (mcd_transformed['category'] == 'Desserts'), 
             ]

values = [3, 2, 1]

mcd_transformed['food_class'] = np.select(conditions, values)

mcd_transformed.head()

In [ ]:
bk_mcd_df.head()

In [ ]:
# Create a filtered dataframe from specific columns
bk_mcd_cols = ["Chain", "Type", "Item", "Saturated Fat (g)", "Calories"]
bk_mcd_transformed= bk_mcd_df[bk_mcd_cols].copy()

# Rename the column headers
bk_mcd_transformed = bk_mcd_transformed.rename(columns={"Type": "category",
                                                    "Item": "item",
                                                    "Saturated Fat (g)": "saturated_fat",
                                                    "Calories": "calories"})

bk_mcd_transformed.head()

In [ ]:
# Find full list of categories
bk_mcd_transformed['category'].unique()

In [ ]:
# manually assign each category to a food_class 
conditions = [(bk_mcd_transformed['category'] == 'Whopper Sandwiches') | (bk_mcd_transformed['category'] == 'Flame Broiled Burgers') \
                  | (bk_mcd_transformed['category'] == 'Chicken & More') | (bk_mcd_transformed['category'] == 'Salads & Sides') \
                  | (bk_mcd_transformed['category'] == 'King Jr Meals - Entrees') | (bk_mcd_transformed['category'] == 'King Jr Meals - Sides') \
                  | (bk_mcd_transformed['category'] == 'Breakfast') | (bk_mcd_transformed['category'] == 'Additional Options') \
                  | (bk_mcd_transformed['category'] == 'Sandwiches') | (bk_mcd_transformed['category'] == 'French Fries') \
                  | (bk_mcd_transformed['category'] == 'Chicken & Sauce') | (bk_mcd_transformed['category'] == 'Salads') \
                  | (bk_mcd_transformed['category'] == 'Salad Dressings'),
              (bk_mcd_transformed['category'] == 'Beverages') | (bk_mcd_transformed['category'] == 'McCafe Coffees') \
                  | (bk_mcd_transformed['category'] == 'King Jr Meals - Beverages') | (bk_mcd_transformed['category'] == 'Shakes/Smoothies') \
                  | (bk_mcd_transformed['category'] == 'Soft Drinks') | (bk_mcd_transformed['category'] == 'Hot Coffees') \
                  | (bk_mcd_transformed['category'] == 'Iced Coffees') | (bk_mcd_transformed['category'] == 'Frappes') \
                  | (bk_mcd_transformed['category'] == 'McCafe Coffees - Nonfat Milk') | (bk_mcd_transformed['category'] == 'McCafe Coffees - Whole Milk') \
                  | (bk_mcd_transformed['category'] == 'McCafe Frappes') | (bk_mcd_transformed['category'] == 'McCafe Smoothies'),
              (bk_mcd_transformed['category'] == 'Desserts') | (bk_mcd_transformed['category'] =='King Jr Meals - Desserts') \
                  | (bk_mcd_transformed['category'] == 'Desserts/Shakes') | (bk_mcd_transformed['category'] =='King Jr Meals - Desserts'), 
             ]

values = [3, 2, 1]

bk_mcd_transformed['food_class'] = np.select(conditions, values)

bk_mcd_transformed.head()

In [ ]:
# remove bad data (namely the  ' -   ' values found in the original csv)
bk_mcd_transformed = bk_mcd_transformed[bk_mcd_transformed['saturated_fat'] != ' -   ']

In [ ]:
# convert , decimal place to . and set to float64 datatype
bk_mcd_transformed['saturated_fat'] = bk_mcd_transformed['saturated_fat'].str.replace(',', '.')
bk_mcd_transformed['saturated_fat'] = bk_mcd_transformed['saturated_fat'].astype('float64')

In [ ]:
# Split bk and mcd into seperate dataframes
bk_transformed =  bk_mcd_transformed.loc[bk_mcd_transformed['Chain'] == 'Burger King']
mcd_2_join_transformed =  bk_mcd_transformed.loc[bk_mcd_transformed['Chain'] == 'Mc Donalds']
mcd_2_join_transformed.head()

In [ ]:
mcd_transformed_combined = mcd_transformed.merge(mcd_2_join_transformed, how = "right")
mcd_transformed_combined

In [ ]:
# Remove the 'Chain' column from the dataframes
mcd_transformed_combined = mcd_transformed_combined.drop(columns=['Chain'])
bk_transformed = bk_transformed.drop(columns=['Chain'])

In [ ]:
mcd_transformed_combined["id"] = mcd_transformed_combined.index
mcd_transformed_combined.set_index("id", inplace=True)
mcd_transformed_combined.head()

In [ ]:
bk_transformed["id"] = bk_transformed.index
bk_transformed.set_index("id", inplace=True)
bk_transformed.head()

### Transform Starbucks Data

In [ ]:
starbucks_food_df.head()

In [ ]:
#starbucks_food_df.dtypes

In [ ]:
# Create a filtered dataframe from specific columns
starbs_food_cols = ["Category", "Name", "Calories", "Saturated Fat(g)"]
starbs_food_transformed= starbucks_food_df[starbs_food_cols].copy()

# Rename the column headers
starbs_food_transformed = starbs_food_transformed.rename(columns={"Category": "category",
                                                                "Name": "item",
                                                                "Saturated Fat(g)": "saturated_fat",
                                                                "Calories": "calories"
                                                                })

# Show transformed db
starbs_food_transformed.head()

In [ ]:
# Add new column for class designator
# Set column value equal to "food_class number 3" designating food for all
starbs_food_transformed["food_class"] = 3

starbs_food_transformed.head()

In [ ]:
#starbs_food_transformed["item"].unique()

In [ ]:
# Update individual item to dessert class if needed
# Assign each category to a food_class manually
conditions = [(starbs_food_transformed["item"] == "Birthday Cake Pop") |
              (starbs_food_transformed["item"] == "Blueberry Oat Cake") |
              (starbs_food_transformed["item"] == "Chocolate Cake Pop") |
              (starbs_food_transformed["item"] == "Chocolate Chip Cookie") |
              (starbs_food_transformed["item"] == "Chocolate Chip Cookie Dough Cake Pop") |
              (starbs_food_transformed["item"] == "Classic Coffee Cake") |
              (starbs_food_transformed["item"] == "Confetti Sugar Cookie") |
              (starbs_food_transformed["item"] == "Double Chocolate Chunk Brownie") |
              (starbs_food_transformed["item"] == "Frosted Doughnut Cake Pop")|
              (starbs_food_transformed["item"] == "Gluten-Free Marshmallow Dream Bar") |
              (starbs_food_transformed["item"] == "Iced Lemon Loaf Cake") |
              (starbs_food_transformed["item"] == "Old-Fashioned Glazed Doughnut") |
              (starbs_food_transformed["item"] == "Strawberry Cake Pop")
             ]

# This is the value for a dessert
values = [1]

starbs_food_transformed['food_class'] = np.select(conditions, values)
        
# Show transformed db    
starbs_food_transformed.head()

In [ ]:
# Change all 0 values back to 3 for food_class
starbs_food_transformed["food_class"].replace(0, 3, inplace = True)
starbs_food_transformed.head()

In [ ]:
# Final look at food db info
starbs_food_transformed.count()

In [ ]:
starbucks_drink_df.head()

In [ ]:
# Create a filtered dataframe from specific columns
starbs_drink_cols = ["Category", "Name", "Calories", "Saturated fat(g)"]
starbs_drink_transformed= starbucks_drink_df[starbs_drink_cols].copy()

# Rename the column headers
starbs_drink_transformed = starbs_drink_transformed.rename(columns={"Category": "category",
                                                                    "Name": "item",
                                                                    "Saturated fat(g)": "saturated_fat",
                                                                    "Calories": "calories"
                                                                    })

# Show transformed db
starbs_drink_transformed.head()

In [ ]:
# Add new column for class designator
# Set column value equal to "food_class number 2" designating drink for all
starbs_drink_transformed["food_class"] = 2

starbs_drink_transformed.head()

In [ ]:
# See all of the different category listings
#starbs_drink_transformed["category"].unique()

In [ ]:
starbs_drink_transformed.count()

In [ ]:
# Join the dataframes to get a starbucks food and drink df
starbs_menu_df = starbs_food_transformed.merge(starbs_drink_transformed, how = "outer")
starbs_menu_df

In [ ]:
# Adjust so saturated_fat column is before calories
mid = starbs_menu_df["saturated_fat"]
starbs_menu_df.drop(labels=["saturated_fat"], axis = 1, inplace = True)
starbs_menu_df.insert(2, "saturated_fat", mid)
starbs_menu_df["id"] = starbs_menu_df.index
starbs_menu_df.set_index("id", inplace=True)
starbs_menu_df

## Create database connection

In [ ]:
# Update Username and Password for pgAdmin
# Also update Database Name to match what you created at the start
connection_string = f"{username}:{password}@localhost:5432/FastFood_db"

# Create the engine
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
# You should see ["McDonalds", "Burger_King", "Starbucks", "Subway", "Food_Classes"]
engine.table_names()

## Load DataFrames into database

In [ ]:
# Use "to_sql" function to load all transformed dfs' data into postgres

# Starbucks
starbs_menu_df.to_sql(name='Starbucks', con=engine, if_exists='append', index=True)

In [ ]:
# Subway
subway_transformed.to_sql(name='Subway', con=engine, if_exists='append', index=True)

In [ ]:
# McDonalds
mcd_transformed_combined.to_sql(name='McDonalds', con=engine, if_exists='append', index=True)

In [ ]:
# Burger King
bk_transformed.to_sql(name='Burger_King', con=engine, if_exists='append', index=True)